# Basic Postgresql

## Init

In [114]:
import os
import psycopg2
import psycopg2.pool
from dotenv import load_dotenv
load_dotenv()

DATABASE_URL = os.getenv("DATABASE_URL")

from prettyprinter import pprint
#pp = pprint.PrettyPrinter(indent=4)


## Automatically Return Connection to Pool

Usually, just using the connection with with statement is enough. It automatically closes the connection the program leaves the with context. However, it doesn't return the connection to its pool. Therefore, I find it better to create another context manager for this. However, I don't want to make a wrapper or subclass of the connection pool, so I will just use this hacky solution where I just insert the function into the connection pool class.

In [112]:
#connection pool hack
from contextlib import contextmanager
@contextmanager
def withconn(self, key=None):
    try:
        conn = self.getconn(key)
        with conn:
            yield conn
    except:
        raise
    finally:
        self.putconn(conn, key)
        
psycopg2.pool.ThreadedConnectionPool.withconn = withconn

## Create Connection Pool

In [113]:
#psycopg2.pool.ThreadedConnectionPool(min_connection, max_connection, database_url)
conn_pool = psycopg2.pool.ThreadedConnectionPool(0, 10, DATABASE_URL)

## List All Tables

Here we use one with statement for two objects: connection and cursor. This is how it's done generally unless you want to have persistent connection (with statement will close it after usage). Keeping persistent connection should only be done if you're sure the server won't close it from their side. If not persistent, connection should be closed if you will not immediately be using it again.

Cursor is used to execute sql and retrieve results. Cursor should be reused if you're going to immediately execute another query. Cursor should be closed after you're done to clear the cached results. Usually the system will take care of the garbage, but this is just good practice because for some database that's not the case.

All tables are listed in pg_catalog.pg_tables, including the catalog itself and the schema info. So to get all "real" tables, we have to exclude those two tables.

Triple quotation marks are used to wrap multiline strings. The semicolon at the end of the query is sign of the end of a sql statement, but usually this is not needed unless you want to execute multiple statements in one go.

In [52]:
def get_table_names():
    with conn_pool.withconn() as conn, conn.cursor() as cur:
        cur.execute("""
            SELECT
                *
            FROM
                pg_catalog.pg_tables
            WHERE
                schemaname != 'pg_catalog'
            AND schemaname != 'information_schema';
        """)
        table_names = [table[1] for table in cur.fetchall()]
        return table_names
    
pprint(get_table_names())

[
    'django_migrations',
    'django_content_type',
    'auth_permission',
    'auth_group',
    'auth_group_permissions',
    'auth_user',
    'auth_user_groups',
    'auth_user_user_permissions',
    'django_admin_log',
    'django_session',
    'web_greeting'
]


## Create Table

The execute function doesn't return anything. cur.rowcount also will always return -1 whether create table succeeds or not. However, if it fails, it will throw an exception. If you want to create table only if it exists, use CREATE TABLE IF NOT EXISTS.

In [69]:
with conn_pool.withconn() as conn, conn.cursor() as cur:
    cur.execute("""
        CREATE TABLE test (id serial PRIMARY KEY, num integer, data varchar);
    """)
    

DuplicateTable: relation "test" already exists


In [73]:
pprint(get_table_names())

[
    'django_migrations',
    'django_content_type',
    'auth_permission',
    'auth_group',
    'auth_group_permissions',
    'auth_user',
    'auth_user_groups',
    'auth_user_user_permissions',
    'django_admin_log',
    'django_session',
    'web_greeting',
    'test'
]


In [70]:
with conn_pool.withconn() as conn, conn.cursor() as cur:
    cur.execute("""
        CREATE TABLE IF NOT EXISTS test (id serial PRIMARY KEY, num integer, data varchar);
    """)
    
pprint(get_table_names())

[
    'django_migrations',
    'django_content_type',
    'auth_permission',
    'auth_group',
    'auth_group_permissions',
    'auth_user',
    'auth_user_groups',
    'auth_user_user_permissions',
    'django_admin_log',
    'django_session',
    'web_greeting',
    'test'
]


## Drop Table

Alternatively, you can drop the table first (yields quite different effect).

In [71]:
with conn_pool.withconn() as conn, conn.cursor() as cur:
    cur.execute("""
        DROP TABLE test;
    """)
    
pprint(get_table_names())

[
    'django_migrations',
    'django_content_type',
    'auth_permission',
    'auth_group',
    'auth_group_permissions',
    'auth_user',
    'auth_user_groups',
    'auth_user_user_permissions',
    'django_admin_log',
    'django_session',
    'web_greeting'
]


In [72]:
with conn_pool.withconn() as conn, conn.cursor() as cur:
    cur.execute("""
        CREATE TABLE test (id serial PRIMARY KEY, num integer, data varchar);
    """)
    
pprint(get_table_names())

[
    'django_migrations',
    'django_content_type',
    'auth_permission',
    'auth_group',
    'auth_group_permissions',
    'auth_user',
    'auth_user_groups',
    'auth_user_user_permissions',
    'django_admin_log',
    'django_session',
    'web_greeting',
    'test'
]


## Alter Table

### View Table Structure

In [76]:
def get_table_structure(table_name):
    with conn_pool.withconn() as conn, conn.cursor() as cur:
        cur.execute("""
            SELECT column_name, data_type, character_maximum_length
            FROM INFORMATION_SCHEMA.COLUMNS 
            WHERE table_name = '%s';
        """ % (table_name,)
        )
        return cur.fetchall()
    
pprint(get_table_structure("test"))

[
    ('id', 'integer', None),
    ('num', 'integer', None),
    ('data', 'character varying', None),
    ('data2', 'character varying', None)
]


### Add Column

Add column is like create table; it throws an error if exists.

In [83]:
with conn_pool.withconn() as conn, conn.cursor() as cur:
    cur.execute("""
        ALTER TABLE test
        ADD COLUMN data2 int;
    """)
    
pprint(get_table_structure("test"))

[
    ('id', 'integer', None),
    ('num', 'integer', None),
    ('data', 'character varying', None),
    ('data2', 'integer', None)
]


### Alter Column

You can change a column's data type. The USING clause is the clause that will process the existing data so you can handle the conversion better. {exp}::{type} is the syntatic sugar to cast an expression to another type. Here it is used to cast ints in the previous data2 column to varchar, so it will fit the new data type.

In [84]:
with conn_pool.withconn() as conn, conn.cursor() as cur:
    cur.execute("""
        ALTER TABLE test
        ALTER COLUMN data2 
            TYPE varchar(50) 
            USING test::VARCHAR;
    """)
    
pprint(get_table_structure("test"))

[
    ('id', 'integer', None),
    ('num', 'integer', None),
    ('data', 'character varying', None),
    ('data2', 'character varying', 50)
]


### Drop Column

In [85]:
with conn_pool.withconn() as conn, conn.cursor() as cur:
    cur.execute("""
        ALTER TABLE test
        DROP COLUMN data2;
    """)
    
pprint(get_table_structure("test"))

[
    ('id', 'integer', None),
    ('num', 'integer', None),
    ('data', 'character varying', None)
]


## Insert Row

We'll use the following function for ease.

In [86]:
def fetch_table(table_name):
    with conn_pool.withconn() as conn, conn.cursor() as cur:
        cur.execute("""
            SELECT * FROM test
        """)
        return cur.fetchall()

fetch_table("test")

[]

We can just execute an insert statement to insert data. cur.rowcount will then return the number of rows inserted. cur.rowcount will be replaced by new value if you execute new query.

In [90]:
with conn_pool.withconn() as conn, conn.cursor() as cur:
    cur.execute("""
        INSERT INTO test(id, num, data) VALUES(1, 2, 'hello')
    """)
    print("Rowcount: " + str(cur.rowcount))
    
pprint(fetch_table("test"))

1
[(1, 2, 'hello')]


Inserting with value of primary key that already exists will throw an error.

In [91]:
with conn_pool.withconn() as conn, conn.cursor() as cur:
    cur.execute("""
        INSERT INTO test(id, num, data) VALUES(1, 2, 'hello')
    """)
    print("Rowcount: " + str(cur.rowcount))
    
pprint(fetch_table("test"))

UniqueViolation: duplicate key value violates unique constraint "test_pkey"
DETAIL:  Key (id)=(1) already exists.


Passing unsanitized data into a query risks sql injection. The best practice is to use prepared statement or parametrized statement/query instead of directly inserting the data into the query string. Of course, this isn't needed if you're sure that the data is safe, for example if it's a program constant. 

We'll use parametrized statement from now on.

In [93]:
with conn_pool.withconn() as conn, conn.cursor() as cur:
    cur.execute("""
        INSERT INTO test(id, num, data) VALUES(%s, %s, %s)
    """, (2, 3, "bye"))
    print("Rowcount: " + str(cur.rowcount))
    
pprint(fetch_table("test"))

1
[(1, 2, 'hello'), (2, 3, 'bye')]


## Update Row

Pretty straight forward. It also returns rowcount as the number of rows updated.

In [94]:
with conn_pool.withconn() as conn, conn.cursor() as cur:
    cur.execute("""
        UPDATE test SET num=num+%s WHERE id=%s
    """, (2, 2))
    print("Rowcount: " + str(cur.rowcount))
    
pprint(fetch_table("test"))

1
[(1, 2, 'hello'), (2, 5, 'bye')]


Different from MySQL, postgres counts rows updated to the same value. Here you see the rows are basically unchanged, because the new values are the same, but the rowcount still returns 1. In MySQL it will return 0 unless you change its settings.

In [95]:
with conn_pool.withconn() as conn, conn.cursor() as cur:
    cur.execute("""
        UPDATE test SET num=%s WHERE id=%s
    """, (5, 2))
    print("Rowcount: " + str(cur.rowcount))
    
pprint(fetch_table("test"))

1
[(1, 2, 'hello'), (2, 5, 'bye')]


## Delete Row

Lastly, we delete rows. It can accept WHERE clause like UDPATE. rowcount returns the number of rows deleted.

In [105]:
with conn_pool.withconn() as conn, conn.cursor() as cur:
    cur.execute("""
        DELETE FROM test
    """)
    print("Rowcount: " + str(cur.rowcount))
    
pprint(fetch_table("test"))

1
[]


## Transaction

By default, autocommit is false. However, when using connection in with statement, it appears that it is automatically commited if succeeds. However, the previous queries should be rolled back if one is error. Let's test that.

In [106]:
with conn_pool.withconn() as conn, conn.cursor() as cur:
    cur.execute("""
        INSERT INTO test(id, num, data) VALUES(%s, %s, %s)
    """, (1, 2, "hello"))
    print("Rowcount: " + str(cur.rowcount))
    cur.execute("""
        INSERT INTO test(id, num, data) VALUES(%s, %s, %s)
    """, (2, 3, "bye"))
    print("Rowcount: " + str(cur.rowcount))
    cur.execute("""
        INSERT INTO test(id, num, data) VALUES(%s, %s, %s)
    """, (2, 3, "bye"))
    print("Rowcount: " + str(cur.rowcount))
    
pprint(fetch_table("test"))

UniqueViolation: duplicate key value violates unique constraint "test_pkey"
DETAIL:  Key (id)=(2) already exists.


In [107]:
pprint(fetch_table("test"))

[]


Cool. Now let's try rolling back intentionally.

In [108]:
with conn_pool.withconn() as conn, conn.cursor() as cur:
    cur.execute("""
        INSERT INTO test(id, num, data) VALUES(%s, %s, %s)
    """, (1, 2, "hello"))
    print("Rowcount: " + str(cur.rowcount))
    cur.execute("""
        INSERT INTO test(id, num, data) VALUES(%s, %s, %s)
    """, (2, 3, "bye"))
    print("Rowcount: " + str(cur.rowcount))
    conn.rollback()
    
pprint(fetch_table("test"))

1
1
[]


Nice. Now let's try commiting before error. The inserts should be saved.

In [117]:
with conn_pool.withconn() as conn, conn.cursor() as cur:
    cur.execute("""
        INSERT INTO test(id, num, data) VALUES(%s, %s, %s)
    """, (1, 2, "hello"))
    print("Rowcount: " + str(cur.rowcount))
    cur.execute("""
        INSERT INTO test(id, num, data) VALUES(%s, %s, %s)
    """, (2, 3, "bye"))
    print("Rowcount: " + str(cur.rowcount))
    conn.commit()
    cur.execute("""
        INSERT INTO test(id, num, data) VALUES(%s, %s, %s)
    """, (2, 3, "bye"))
    print("Rowcount: " + str(cur.rowcount))
    
pprint(fetch_table("test"))

Rowcount: 1
Rowcount: 1


UniqueViolation: duplicate key value violates unique constraint "test_pkey"
DETAIL:  Key (id)=(2) already exists.


In [110]:
pprint(fetch_table("test"))

[(1, 2, 'hello'), (2, 3, 'bye')]


## Unpacking Fetched Tuple

This is python only, and it's quite handy. So you can unpack a tuple like:

var1, var2 = (1, 2)

And you can use that for every fetched row. Neat.

In [118]:
for id, num ,data in fetch_table("test"):
    print("id=%d, num=%d, and data='%s'" % (id, num, data))

id=1, num=2, and data='hello'
id=2, num=3, and data='bye'


This concludes the basic psycopg2 notebook. We did not and will not cover joins and stuff because it's more of sql (or postgresql) rather than psycopg2.